In [1]:
import sys
print(sys.executable)
print(sys.version)

/opt/homebrew/opt/python@3.12/bin/python3.12
3.12.4 (main, Jun  6 2024, 18:26:44) [Clang 15.0.0 (clang-1500.3.9.4)]


In [22]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.cluster import HDBSCAN
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [3]:
data = pd.read_parquet('proc_data')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8636 entries, C10001 to C19190
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   BALANCE                           8636 non-null   float64
 1   BALANCE_FREQUENCY                 8636 non-null   float64
 2   PURCHASES                         8636 non-null   float64
 3   ONEOFF_PURCHASES                  8636 non-null   float64
 4   INSTALLMENTS_PURCHASES            8636 non-null   float64
 5   CASH_ADVANCE                      8636 non-null   float64
 6   PURCHASES_FREQUENCY               8636 non-null   float64
 7   ONEOFF_PURCHASES_FREQUENCY        8636 non-null   float64
 8   PURCHASES_INSTALLMENTS_FREQUENCY  8636 non-null   float64
 9   CASH_ADVANCE_FREQUENCY            8636 non-null   float64
 10  CASH_ADVANCE_TRX                  8636 non-null   int64  
 11  PURCHASES_TRX                     8636 non-null   int64  
 12  CRED

In [4]:
tsne_columns = [f"tsne{i+1}" for i in range(3)]
pca_columns = [f"pca{i+1}" for i in range(3)]
pca_tsne_cols = tsne_columns + pca_columns

In [15]:
cat_features = []
num_features = [col for col in data.columns if col not in pca_tsne_cols]
features = cat_features + num_features
print(f"{len(cat_features)=}")
print(f"{len(num_features)=}")
print(f"{len(features)=}")

len(cat_features)=0
len(num_features)=17
len(features)=17


In [66]:
data.shape

(8636, 23)

In [67]:
hdbscan_data = data.copy()

In [111]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num_features', StandardScaler(), num_features),
        ('cat_features', OneHotEncoder(), cat_features),
    ]
)
reducer = TSNE(n_components=3)
clusterer = HDBSCAN(min_cluster_size=100, min_samples=1)

pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        # ('reducer', reducer),
        ('clusterer', clusterer, ),
    ]
)
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_features',
                                                  StandardScaler(),
                                                  ['BALANCE',
                                                   'BALANCE_FREQUENCY',
                                                   'PURCHASES',
                                                   'ONEOFF_PURCHASES',
                                                   'INSTALLMENTS_PURCHASES',
                                                   'CASH_ADVANCE',
                                                   'PURCHASES_FREQUENCY',
                                                   'ONEOFF_PURCHASES_FREQUENCY',
                                                   'PURCHASES_INSTALLMENTS_FREQUENCY',
                                                   'CASH_ADVANCE_FREQUENCY',
                                                   'CASH_ADVANCE_TRX',
                                                   'PURCHASES_TRX',
                                                   'CREDIT_LIMIT', 'PAYMENTS',
                                                   'MINIMUM_PAYMENTS',
                                                   'PRC_FULL_PAYMENT',
                                                   'TENURE']),
                                                 ('cat_features',
                                                  OneHotEncoder(), [])])),
                ('clusterer', HDBSCAN(min_cluster_size=100, min_samples=1))])

In [112]:
%%time
cluster = pipeline.fit_predict(hdbscan_data[features])
hdbscan_data['cluster'] = cluster

CPU times: user 370 ms, sys: 1.92 ms, total: 372 ms
Wall time: 376 ms


In [113]:
len(cluster), len(set(cluster))

(8636, 8)

In [114]:
hdbscan_data.cluster.value_counts()

cluster
-1    7141
 6     320
 5     287
 2     254
 4     181
 1     172
 3     169
 0     112
Name: count, dtype: int64